In [8]:
library(Seurat)
library(SeuratDisk)
library(NMF)
library(ggalluvial)
library(ComplexHeatmap)
library(CellChat)
library(patchwork)
library(graphics)
library(Matrix)
library(circlize)
library(colorspace)
library(pracma)
library(glue)

options(stringsAsFactors = FALSE, repr.plot.width = 12, repr.plot.height = 9, repr.plot.res = 300)

source("B-cellType_pathways-heatmap.r")

# Parameters

In [238]:
# for plotting figC, differential network
disease = "MDD"
project = "PTSD_Call"
type <- "truncatedMean"
trim <- 0.05

cell_type <- "MG"
comm_type <- "sender"       # "receiver", "sender"

In [239]:
celltype_name = c('Astro', 'Endo', 'MG', 'Oligo', 'OPC', 
                  'Exc CUX2', 'Exc FEZF2', 'Exc OPRK1', 'Exc RORB', 
                  'Inh KCNG1', 'Inh LAMP5', 'Inh PVALB', 'Inh SST', 'Inh VIP')
measure = 'weight'

# Read in Data

In [169]:
cellchat.CT <- readRDS(file = glue("/gpfs/gibbs/pi/girgenti/JZhang/CL/C2C/PTSD_Call/Obj-CellChat/RNA_FINAL-CON-type_{type}-trim_{trim}_cellchat.rds"))
cellchat.DIS <- readRDS(file = glue("/gpfs/gibbs/pi/girgenti/JZhang/CL/C2C/PTSD_Call/Obj-CellChat/RNA_FINAL-{disease}-type_{type}-trim_{trim}_cellchat.rds"))

In [170]:
object.list <- list(CT = cellchat.CT, DIS = cellchat.DIS)
cellchat <- mergeCellChat(object.list, add.names = names(object.list))
print(cellchat)
names(object.list)

Merge the following slots: 'data.signaling','images','net', 'netP','meta', 'idents', 'var.features' , 'DB', and 'LR'.



An object of class CellChat created from a merged object with multiple datasets 
 616 signaling genes.
 625220 cells. 
CellChat analysis of single cell RNA-seq data! 


[1] "CT"  "DIS"

# Code

## Normalization

In [188]:
# get the matrix
obj1 <- object.list[[1]]@net[[measure]]              # CT
obj2 <- object.list[[2]]@net[[measure]]              # DIS
melt_1 <- reshape2::melt(obj1, value.name="count")
melt_2 <- reshape2::melt(obj2, value.name="count")
sum1 <- sum(melt_1$count)
sum2 <- sum(melt_2$count)

# balance
normalization = sum1 / sum2
# normalization = sum(object.list[[1]]@netP$prob) / sum(object.list[[2]]@netP$prob)
net.diff <- obj2 * (normalization) - obj1

# reorder the celltypes
net.diff <- net.diff[celltype_name, celltype_name]

In [189]:
colSums(net.diff)

Astro       Endo         MG      Oligo        OPC   Exc CUX2  Exc FEZF2 
-0.5634878 -0.6126809 -0.6068820  0.3373892  0.3878068 -0.2903301 -0.5447684 
 Exc OPRK1   Exc RORB  Inh KCNG1  Inh LAMP5  Inh PVALB    Inh SST    Inh VIP 
-0.2344671 -0.4568127  0.9379691  0.9650177  1.1558522 -0.9751247  0.5005185

In [190]:
sum(net.diff_comm_slice)

[1] -0.2554606

## C2C Difference

In [240]:
data_whole.CT  <- cellchat.CT@netP$prob
data_whole.DIS <- cellchat.DIS@netP$prob * normalization

if (comm_type == "receiver") {
    comm_slice.CT  <- data_whole.CT[, cell_type, ]
    comm_slice.DIS <- data_whole.DIS[, cell_type, ]
    row_title = paste0("Sender Cell Types to ", cell_type)
} else if (comm_type == "sender") {
    comm_slice.CT  <- data_whole.CT[cell_type, , ]
    comm_slice.DIS <- data_whole.DIS[cell_type, , ]
    row_title = paste0(cell_type, " to Receiver Cell Types")
}

if (length(setdiff(colnames(comm_slice.CT), colnames(comm_slice.DIS))) > 0) {
    original_names <- colnames(comm_slice.DIS)
    for (i in 1:length(setdiff(colnames(comm_slice.CT), colnames(comm_slice.DIS)))) {
        comm_slice.DIS <- cbind(comm_slice.DIS, 0)
    }
    colnames(comm_slice.DIS) <- c(original_names, setdiff(colnames(comm_slice.CT), colnames(comm_slice.DIS)))
}

if (length(setdiff(colnames(comm_slice.DIS), colnames(comm_slice.CT))) > 0) {
    original_names <- colnames(comm_slice.CT)
    for (i in 1:length(setdiff(colnames(comm_slice.DIS), colnames(comm_slice.CT)))) {
        comm_slice.CT <- cbind(comm_slice.CT, 0)
    }
    colnames(comm_slice.CT) <- c(original_names, setdiff(colnames(comm_slice.DIS), colnames(comm_slice.CT)))
}

comm_slice.DIS <- comm_slice.DIS[, colnames(comm_slice.CT)]

net.diff_comm_slice <- comm_slice.DIS - comm_slice.CT
net.diff_comm_slice <- net.diff_comm_slice[celltype_name, abs(colSums(net.diff_comm_slice)) > 0.005]
net.diff_comm_slice <- net.diff_comm_slice[celltype_name, names(sort(colSums(net.diff_comm_slice), decreasing = TRUE))]

# Visual

In [241]:
# Draw
# pdf(file = glue("/gpfs/gibbs/project/girgenti/cl2553/C2C/PTSD_Call/figures/{disease}vsCON/{type}-{trim}/supp_fig-diff_IO-{comm_type}-{cell_type}.pdf"), 
#     width = 14, height = 8)
pdf(file = "image.pdf", width = 8, height = 8)
ht1 <- Heatmap(net.diff_comm_slice, 
              name = "Diff Strength", #title of legend
              column_title = glue("Differential {disease} vs CON Pathways"), row_title = row_title, 
              column_title_gp = gpar(fontsize = 16, fontface="bold"),                                      # Column Title Font
              row_title_gp = gpar(fontsize = 16, fontface="bold"),                                                            # Row Title Font
              row_names_gp = gpar(fontsize = 12), column_names_gp = gpar(fontsize = 12),
              show_row_dend = FALSE, show_column_dend = FALSE,                                               # No dendrograms
              rect_gp = gpar(col = "white", lwd = 2),                                                        # ea cell has a white border
              col = colorRamp2(c(-max(abs(net.diff_comm_slice)), 0, max(abs(net.diff_comm_slice))), 
                                c("blue", "white", "red")),     # Set Color
              row_names_side = 'left', column_names_side = 'top', 
              heatmap_legend_param = list(legend_direction = "horizontal", legend_width = unit(5, "cm")),
              width = ncol(net.diff_comm_slice)*unit(5, "mm"), height = nrow(net.diff_comm_slice)*unit(8, "mm"),   # Row and column names on the right side
              cluster_rows = FALSE, cluster_columns = TRUE)
draw(ht1, heatmap_legend_side = "bottom")
dev.off()

png 
  2